# 🎯 Emotion-Domain Classification with PyCaret AutoML (`domain_EI`, 4 classes)

This notebook builds ML classifiers to predict the **emotion-intelligence domain** (`domain_EI`) with four labels  
(HAHV · HALV · LALV · LAHV  →  encoded as 0-3) using **PyCaret 3**.

---

## 🧩 Objectives

| Item | Details |
|------|---------|
| **Feature sets** | 4 configurations (score_EI / group_EI included or removed) |
| **Scaling options** | `none`, `standard`, `minmax` |
| **Model family** | CatBoost · XGBoost · Random Forest · Extra Trees · Logistic Regression · Naive Bayes · SVM · K-NN · QDA |
| **Total runs** | **12** (4 feature configs × 3 scalers) |
| **Validation** | Stratified **3-fold CV** inside PyCaret + **20 % hold-out** |
| **Artifacts** | PNG plots (AUC, confusion matrix, feature importance) + CSV summaries |

---

## 🛠️ Pipeline Overview

| Step | Description |
|------|-------------|
| **1 — Split** | Apply feature-exclusion rules, then stratified 80 / 20 train-test split |
| **2 — Scaling** | Transform numeric features with the chosen scaler (`none` / `StandardScaler` / `MinMaxScaler`) |
| **3 — AutoML** | `compare_models()` evaluates the candidate models via CV and returns the best estimator |
| **4 — Hold-out Evaluation** | Predict on the reserved test set; save confusion-matrix & AUC plots |
| **5 — Result Logging** | Collect CV AUC / Accuracy and hold-out Accuracy for each run into a master CSV |

---

## 📂 Output Artifacts

| File / Folder | Contents |
|---------------|----------|
| **`overall_results.csv`** | One-row summary per run (feature cfg · scaler · chosen model · metrics) |
| **`leaderboard_cfg*_*.csv`** | Full PyCaret leaderboard (all models, CV statistics) |
| **`CM_/`, `AUC_/`, `FI_/`** | Confusion-matrix, AUC curve and feature-importance PNGs named by run tag |
| *(optional)* `best_models.pkl` | Serialized best estimators via `joblib.dump()` |

---

## 🔧 Requirements

* **PyCaret 3.x** (classification module)  
* `scikit-learn`, `pandas`, `numpy`, `matplotlib`, `tqdm`  
* CatBoost & XGBoost are pulled automatically with the full PyCaret install

> **Note**  
> * All stimulus-related (`Arousal`, `Valence`, etc.) and subjective self-report variables are **permanently excluded** from the feature space.  
> * The four feature configurations are designed to measure the impact of **EI score** (`score_EI`) and **EI group label** (`group_EI`) on model performance.  
> * GPU acceleration, SHAP explanations and 5-fold CV are **not** used in the current pipeline; they can be added later if needed.


## Import Libraries

In [1]:
# ---------------------------------------------------------------
#  ❖  CONFIG & PREP
# ---------------------------------------------------------------
import os, sys, types, re, joblib, warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier          # used for FS
from pycaret.classification import *
warnings.filterwarnings("ignore", category=UserWarning)

from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Step 0: Define Save Paths

We define clean, relative paths for all outputs such as feature importance plots, evaluation plots, and CSV results.

In [2]:
# Base directory: project root (current notebook folder)
ROOT_PATH = os.getcwd()
RES_PATH = os.path.join(ROOT_PATH, '../res')

# DATA
DATA_PATH = os.path.join(os.getcwd(), '../data/')
EXCEL_PATH = "../data/ETRI_cardiac.xlsx"
CSV_PATH = "../data/ETRI_cardiac.csv"
DF_PATH = "../data/updated_data.csv"

# Define subfolders for saving results
FI_PATH   = os.path.join(RES_PATH, 'FI_plots')      # Feature Importance
CM_PATH   = os.path.join(RES_PATH, 'CM_plots')      # Confusion Matrix
AUC_PATH  = os.path.join(RES_PATH, 'AUC_plots')     # AUC Curves
SHAP_PATH = os.path.join(RES_PATH, 'SHAP_plots')    # SHAP Plots
RES_CSV_PATH  = os.path.join(RES_PATH, 'results_csv')   # CSV files

# Create directories if they don't exist
for path in [FI_PATH, CM_PATH, AUC_PATH, SHAP_PATH, RES_CSV_PATH]:
    os.makedirs(path, exist_ok=True)

In [3]:
df = pd.read_csv(DF_PATH)
df.head()

,name,year,score_EI,domain_EI,trainnig,group_EI,자극_Arousal,자극_Valence,Arousal,Valence,...,VLF/HF_autocorr,LF/HF_autocorr,tPow_autocorr,dPow_autocorr,dHz_autocorr,pPow_autocorr,pHz_autocorr,CohRatio_autocorr,RSA_autocorr,dHz_diff_autocorr
0,subj_01_00,2021,400,0,0,0,1,1,4,5,...,-0.567119,-0.991761,4.761754,-0.106286,-0.597611,-0.177651,-1.020465,-1.087973,1.085212,-0.595462
1,subj_01_01,2021,400,1,0,0,1,0,6,4,...,-2.854562,-1.532805,-4.013737,13.465573,-1.486156,0.136173,-0.504705,-0.481968,0.855739,-1.486156
2,subj_01_02,2021,400,3,0,0,0,1,3,6,...,-1.375726,-1.030548,0.056760,-1.111399,-1.126756,-0.961825,-1.626310,-1.110674,-0.670813,-1.127433
3,subj_01_03,2021,400,2,0,0,0,0,4,4,...,-0.198272,0.921968,-2.812514,7.058668,-0.880433,-0.866042,-0.602454,-1.700471,0.168826,-0.880433
4,subj_02_00,2021,450,0,0,0,1,1,6,7,...,0.469754,0.216407,0.900575,0.121397,0.023057,-0.702703,-0.874824,0.579896,-0.289846,0.023057


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 100 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               180 non-null    object 
 1   year               180 non-null    int64  
 2   score_EI           180 non-null    int64  
 3   domain_EI          180 non-null    int64  
 4   trainnig           180 non-null    int64  
 5   group_EI           180 non-null    int64  
 6   자극_Arousal         180 non-null    int64  
 7   자극_Valence         180 non-null    int64  
 8   Arousal            180 non-null    int64  
 9   Valence            180 non-null    int64  
 10  Subj_AR            180 non-null    int64  
 11  Subj_PN            180 non-null    int64  
 12  BPM                180 non-null    float64
 13  SDNN               180 non-null    float64
 14  rMSSD              180 non-null    float64
 15  VLF                180 non-null    float64
 16  LF                 180 no

## Step 1: Experiment Settings
Define Scaling and Setup for PyCaret 

We define three types of scaling: none, standard, and min-max, for every feature configuration.

In [ ]:
TARGET = "domain_EI"

# Columns that are never used as features
MUST_EXCLUDE = [
    "name", "domain_EI", "year", "trainnig",
    "Arousal", "Valence", "자극_Arousal", "자극_Valence",
    "Subj_AR", "Subj_PN"
]

# 4 feature-set flavours: keep / drop EI score / drop EI group / drop both
FEATURE_EXCLUDE_SETS = {
    1: [],
    2: ["group_EI"],
    3: ["score_EI"],
    4: ["score_EI", "group_EI"],
}

# Three scaler options
SCALERS = {
    "none":     None,
    "standard": StandardScaler(),
    "minmax":   MinMaxScaler(),
}

# Models we allow PyCaret to try
INCLUDE_MODELS = [
    "catboost", "xgboost", "rf", "et",
    "lr", "nb", "svm", "knn", "qda",
]

'TARGET = "domain_EI"\n\n# Columns that are never used as features\nMUST_EXCLUDE = [\n    "name", "domain_EI", "year", "trainnig",\n    "Arousal", "Valence", "자극_Arousal", "자극_Valence",\n    "Subj_AR", "Subj_PN"\n]\n\n# 4 feature-set flavours: keep / drop EI score / drop EI group / drop both\nFEATURE_EXCLUDE_SETS = {\n    1: [],\n    2: ["group_EI"],\n    3: ["score_EI"],\n    4: ["score_EI", "group_EI"],\n}\n\n# Three scaler options\nSCALERS = {\n    "none":     None,\n    "standard": StandardScaler(),\n    "minmax":   MinMaxScaler(),\n}\n\n# Models we allow PyCaret to try\nINCLUDE_MODELS = [\n    "catboost", "xgboost", "rf", "et",\n    "lr", "nb", "svm", "knn", "qda",\n]'

####  LightGBM stub

In [7]:
lgb_stub = types.ModuleType("lightgbm")

class _DummyLGBM:
    """Minimal fake LightGBM estimator so that PyCaret can import it."""
    def __init__(self, *args, **kwargs): pass
    def fit(self, *args, **kwargs): return self
    def predict(self, X, *args, **kwargs):          # label output
        return np.zeros(len(X), dtype=int)
    def predict_proba(self, X, *args, **kwargs):    # proba output
        # return 2-col dummy probs for binary; PyCaret only checks shape
        return np.zeros((len(X), 2), dtype=float)

# expose expected symbols at top level
lgb_stub.LGBMClassifier = _DummyLGBM
lgb_stub.LGBMRegressor  = _DummyLGBM
lgb_stub.Dataset        = object          # rarely used by PyCaret

basic_stub = types.ModuleType("lightgbm.basic")
basic_stub.LightGBMError = RuntimeError    # any Exception subclass works

# make `from lightgbm.basic import LightGBMError` succeed
sys.modules["lightgbm.basic"] = basic_stub

# attach sub-module to the parent package stub
lgb_stub.basic = basic_stub

sys.modules["lightgbm"] = lgb_stub

## Step 2. Result Containers

In [8]:
results:     list[dict]        = []     # one row per config+scaler
best_models: dict[str, object] = {}     # tag ➜ fitted model

## Step 3: AutoML Training using PyCaret + tqdm Progress Tracking
WITHOUT OUTLIER vs WITH OUTLIER !! (With Outlier first)

We train models using PyCaret with GPU acceleration, over 8 × 3 configuration combinations.

In [9]:
'''
for config_id, exclude_cols in tqdm(feature_exclude_sets.items(), desc="Feature Configs", position=0):
    for scaler_name, scaler in tqdm(scalers.items(), desc=f"Scalers for Config {config_id}", leave=False, position=1):
        df_exp = df.copy()
        all_excluded = must_exclude + exclude_cols
        
        remaining_cols = [col for col in df_exp.columns if col not in all_excluded]
        if len(remaining_cols) == 0:
            print(f"⚠️ Skipping config {config_id} + scaler {scaler_name} – no features left.")
            continue

        X = df_exp.drop(columns=all_excluded)
        y = df_exp[target]

        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

        # ✅ Feature Check
        print(f"\n📌 Config {config_id} | Scaler: {scaler_name}")
        print(f"🔸 X_train shape: {X_train.shape}")
        print(f"🔸 Feature columns: {list(X_train.columns)}")
        print(f"🔸 Null values:\n{X_train.isnull().sum()}")
        print(f"🔸 Class distribution:\n{y_train.value_counts()}")
        print("-" * 60)
'''

'\nfor config_id, exclude_cols in tqdm(feature_exclude_sets.items(), desc="Feature Configs", position=0):\n    for scaler_name, scaler in tqdm(scalers.items(), desc=f"Scalers for Config {config_id}", leave=False, position=1):\n        df_exp = df.copy()\n        all_excluded = must_exclude + exclude_cols\n        \n        remaining_cols = [col for col in df_exp.columns if col not in all_excluded]\n        if len(remaining_cols) == 0:\n            print(f"⚠️ Skipping config {config_id} + scaler {scaler_name} – no features left.")\n            continue\n\n        X = df_exp.drop(columns=all_excluded)\n        y = df_exp[target]\n\n        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)\n\n        # ✅ Feature Check\n        print(f"\n📌 Config {config_id} | Scaler: {scaler_name}")\n        print(f"🔸 X_train shape: {X_train.shape}")\n        print(f"🔸 Feature columns: {list(X_train.columns)}")\n        print(f"🔸 Null values:\n{X_train.isnull().sum()}"

### Step 2.3. Experiment loop

In [ ]:
for cfg_id, excl_cols in tqdm(FEATURE_EXCLUDE_SETS.items(),
                              desc="Feature Configs"):

    for scaler_name, scaler in tqdm(SCALERS.items(),
                                    desc=f"Scalers for {cfg_id}",
                                    leave=False):

        # 3-1 ▶ split -------------------------------------------------------
        X = df.drop(columns=MUST_EXCLUDE + excl_cols)
        y = df[TARGET]
        X_tr, X_te, y_tr, y_te = train_test_split(
            X, y, stratify=y, test_size=0.10, random_state=42
        )

        if X_tr.shape[1] == 0:             # empty feature guard
            print(f"🚫 cfg{cfg_id}|{scaler_name}: no features"); continue

        # 3-2 ▶ optional scaling -------------------------------------------
        if scaler:
            X_tr = pd.DataFrame(scaler.fit_transform(X_tr),
                                index=X_tr.index, columns=X_tr.columns)
            X_te = pd.DataFrame(scaler.transform(X_te),
                                index=X_te.index, columns=X_te.columns)

        # 3-3 ▶ PyCaret setup  --------------------------------------------
        train_df = pd.concat([X_tr.reset_index(drop=True),
                              y_tr.reset_index(drop=True)], axis=1)
        test_df  = pd.concat([X_te.reset_index(drop=True),
                              y_te.reset_index(drop=True)], axis=1)

        setup(
            data=train_df,
            test_data=test_df,     # 👈 pass *your* hold-out here
            target=TARGET,
            session_id=42,
            fold=3,
            use_gpu=False,
            html=True,
            verbose=True,
            feature_selection=False,
            index=False,
        )

        # 3-4 ▶ model search  ---------------------------------------------
        best = compare_models(include=INCLUDE_MODELS,
                              sort="AUC",   fold=3,
                              turbo=False, verbose=True)

        tag = f"cfg{cfg_id}_{scaler_name}"
        best_models[tag] = best

        # leaderboard CSV ---------------------------------------------------
        pull().assign(config=cfg_id, scaler=scaler_name) \
              .to_csv(os.path.join(RES_CSV_PATH, f"leaderboard_{tag}.csv"),
                      index=False)

        # 3-5 ▶ predict on hold-out & plots -------------------------------
        pred = predict_model(best)     # uses test_data provided in setup()

        plot_model(best, plot="confusion_matrix",   save=True)
        os.replace("Confusion Matrix.png",
                   os.path.join(CM_PATH,  f"CM_{tag}.png"))

        plot_model(best, plot="auc",                 save=True)
        os.replace("AUC.png",
                   os.path.join(AUC_PATH, f"AUC_{tag}.png"))

        # 3-6 ▶ feature importance ----------------------------------------
        fi_path = os.path.join(FI_PATH, f"FI_{tag}.png")

        def save_fi_tree():
            plot_model(best, plot="feature", save=True)
            os.replace("Feature Importance.png", fi_path)

        if hasattr(best, "feature_importances_"):
            save_fi_tree()

        elif best.__class__.__name__.startswith("CatBoost"):
            fi = best.get_feature_importance()
            top = np.argsort(fi)[::-1][:20]
            plt.figure(figsize=(6, 4))
            plt.barh(range(len(top)), fi[top][::-1])
            plt.yticks(range(len(top)), X_tr.columns[top][::-1])
            plt.tight_layout(); plt.savefig(fi_path); plt.close()

        # 3-7 ▶ collect numeric summary -----------------------------------
        cv_metrics = pull().iloc[0]        # row 0 = best model’s CV stats
        results.append({
            "config":      cfg_id,
            "scaler":      scaler_name,
            "model":       best.__class__.__name__,
            "AUC_CV":      cv_metrics["AUC"],
            "Acc_CV":      cv_metrics["Accuracy"],
            "Acc_holdout": (pred[TARGET] == pred["prediction_label"]).mean(),
        })


Feature Configs:   0%|          | 0/4 [00:00<?, ?it/s]

,Description,Value
0,Session id,42
1,Target,Arousal
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6"
4,Original data shape,"(180, 91)"
5,Transformed data shape,"(180, 91)"
6,Transformed train set shape,"(162, 91)"
7,Transformed test set shape,"(18, 91)"
8,Numeric features,90
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.3272,0.5576,0.3272,0.2578,0.2747,0.1371,0.1458,0.2300
et,Extra Trees Classifier,0.2654,0.5520,0.2654,0.2207,0.2381,0.0717,0.0732,0.2167
xgboost,Extreme Gradient Boosting,0.2346,0.5457,0.2346,0.2223,0.2237,0.0518,0.0529,0.3367
catboost,CatBoost Classifier,0.2593,0.5374,0.2593,0.2005,0.2216,0.0578,0.0598,5.6000
nb,Naive Bayes,0.1049,0.4933,0.1049,0.1310,0.1097,-0.0621,-0.0636,0.1933
knn,K Neighbors Classifier,0.1543,0.4606,0.1543,0.1214,0.1294,-0.0436,-0.0455,0.1967
lr,Logistic Regression,0.1975,0.0000,0.1975,0.1998,0.1949,0.0134,0.0134,0.2267
svm,SVM - Linear Kernel,0.2037,0.0000,0.2037,0.1758,0.1525,0.0192,0.0325,0.1967
qda,Quadratic Discriminant Analysis,0.2037,0.0000,0.2037,0.0937,0.1160,-0.0212,-0.0179,0.1833


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.1111,0.5281,0.1111,0.0494,0.0684,-0.1250,-0.1369


,Description,Value
0,Session id,42
1,Target,Arousal
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6"
4,Original data shape,"(180, 91)"
5,Transformed data shape,"(180, 91)"
6,Transformed train set shape,"(162, 91)"
7,Transformed test set shape,"(18, 91)"
8,Numeric features,90
9,Preprocess,True


...

Feature Configs:  75%|███████▌  | 3/4 [03:05<01:00, 60.26s/it]

,Description,Value
0,Session id,42
1,Target,Arousal
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6"
4,Original data shape,"(180, 89)"
5,Transformed data shape,"(180, 89)"
6,Transformed train set shape,"(162, 89)"
7,Transformed test set shape,"(18, 89)"
8,Numeric features,88
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:14:56
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/41 [00:00<?, ?it/s]

## Step 4. Save overall summary

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(RES_PATH,"overall_results.csv"), index=False)

## 🎯 Selective Outlier Removal Based on XGBoost Feature Importance

Instead of removing all outliers across 88 features, we focus only on the top 10 features identified as most influential for classifying `domain_EI` using XGBoost.

### Method

- Extract top 10 features with highest importance scores.
- Apply Z-score based outlier detection (`|Z| > 4`) on those features only.
- Remove rows where extreme values are detected in these features.
- Visualize before/after distributions to confirm effective filtering.

This approach ensures:
- High-impact errors are removed.
- Valuable data is preserved from non-influential features.


In [ ]:
'''# Step1: Important Features
important_features = [
    'HF', 'lnLF', 'RSA_var', 'LFp_var', 'lnVLF_mssd',
    'HF_var', 'VLFp_mssd', 'rMSSD_autocorr', 'dHz_mssd', 'BPM_autocorr'
]
'''

In [ ]:
'''# Step 2. Z-score calculation (Top-10 Imporatant features only) and Z > 4 row detection
z_scores = df[important_features].apply(zscore)

extreme_mask = (np.abs(z_scores) > 4).any(axis=1)
df_outliers = df[extreme_mask]
df_cleaned = df[~extreme_mask].copy()

# Step 3. Remove outliers from the original DataFrame
print(f"⚠️ Rows with Z > 4 in important features: {extreme_mask.sum()} / {len(df)}")
print("Exclude row's name + index:")
display(df_outliers[['name']].reset_index())

print(f"✅ Cleaned data shape: {df_cleaned.shape}")
'''

In [ ]:
'''# Step 4. Plotting the distribution of important features before and after removing outliers
df_before = df[important_features].copy()
df_before['source'] = 'Before'

df_after = df_cleaned[important_features].copy()
df_after['source'] = 'After'

df_plot = pd.concat([df_before, df_after])
df_plot = df_plot.melt(id_vars='source', var_name='feature', value_name='value')

plt.figure(figsize=(20,15))
sns.boxplot(data=df_plot, x='feature', y='value', hue='source')
plt.title("Top Features Distribution: Before vs After Z > 4 Removal")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()'''